In [567]:
import pandas as pd
import numpy as np
import re
import json

In [568]:
dataFile = 'tmp/speciesdata.csv'
df = pd.read_csv(dataFile)
df = df[df['incluir_guia_web']==1]
#df.info()
df.head()

,Família,Espécie,incluir_guia_web,Texto artigo,Registro Guia,Registro Literatura,Fonte literatura,Registro novo,Regist. Guia,Literatura,...,Atividade (noturno/diurno),Tamanho real (mm) - fêmea,Tamanho real (mm) - macho,Meses de Ocorrência,meses_ocorrencia,Chances de encontros (muito raro/raro/frequente/muito frequente),DOI artigo descrição espécie,Link do Artigo de Descrição,Distribuição - SITE REUBER (espécies do DF),Ameaças - SITE REUBER (espécies do DF)
0,AROMOBATIDAE (sapos-foguete),"Allobates goianus (Bokermann, 1975)",1,ok,-,X,"Santoro & Brandão, 2018",-,-,X,...,diurno,16.8-17,16.8-17,chuva,out-mar,Muito raro,10.13128/Acta_Herpetol-17491,https://scholar.google.com.br/scholar?hl=en&as...,-,-
1,BUFONIDAE (sapos-cururu),"Rhaebo guttatus (Schneider, 1799)",1,ok,-,X,"Santoro & Brandão, 2018",-,-,X,...,noturno,174,121,NaN,NaN,Raro,só pdf,NaN,-,-
2,BUFONIDAE (sapos-cururu),"Rhinella mirandaribeiroi (Gallardo, 1965)",1,ok,X,X,"Santoro & Brandão, 2018",-,X,-,...,noturno,49-73,40-71,"janeiro, fevereiro, setembro e outubro","jan-fev, set-out",Raro,0.11606/issn.2176-7793.v40i1p1-73,https://scihub.to/10.11606/issn.2176-7793.v40i...,-,-
3,BUFONIDAE (sapos-cururu),"Rhinella ocellata (Günther, 1858)",1,ok,X,NaN,NaN,NaN,X,-,...,noturno,44,44,chuva até março,out-mar,Frequente,https://doi.org/10.1016/j.ympev.2006.12.006,http://kops.uni-konstanz.de/bitstream/handle/1...,"Tocantins, Para´, Mato Grosso, Minas Gerais, a...",-
4,BUFONIDAE (sapos-cururu),"Rhinella rubescens (Lutz, 1925)",1,ok,X,X,"Santoro & Brandão, 2018",-,X,-,...,noturno,90,90,ano todo,jan-dez,Frequente,NaN,NaN,Espécie com ampla distribuição associada ao bi...,Desmatamentos e remoção de hábitats constituem...


In [569]:
df.rename( {'Espécie':'scientificName',
            'Família':'family',
            'Nome Comum': 'vernacularNames',
            'Fitofisionomias': 'phytos',
            'Postura dos ovos (habitat breeding)*': 'habitat_breeding',
            'Habitat de vida (mata ou floresta/area aberta/pedras)*':'habitat',
            'Tamanho real (mm) - fêmea':'tamanho_femea',
            'Tamanho real (mm) - macho':'tamanho_macho',
            'Chances de encontros (muito raro/raro/frequente/muito frequente)': 'detectability'}, axis=1, inplace=True)

In [570]:
df.head(3)

,family,scientificName,incluir_guia_web,Texto artigo,Registro Guia,Registro Literatura,Fonte literatura,Registro novo,Regist. Guia,Literatura,...,Atividade (noturno/diurno),tamanho_femea,tamanho_macho,Meses de Ocorrência,meses_ocorrencia,detectability,DOI artigo descrição espécie,Link do Artigo de Descrição,Distribuição - SITE REUBER (espécies do DF),Ameaças - SITE REUBER (espécies do DF)
0,AROMOBATIDAE (sapos-foguete),"Allobates goianus (Bokermann, 1975)",1,ok,-,X,"Santoro & Brandão, 2018",-,-,X,...,diurno,16.8-17,16.8-17,chuva,out-mar,Muito raro,10.13128/Acta_Herpetol-17491,https://scholar.google.com.br/scholar?hl=en&as...,-,-
1,BUFONIDAE (sapos-cururu),"Rhaebo guttatus (Schneider, 1799)",1,ok,-,X,"Santoro & Brandão, 2018",-,-,X,...,noturno,174,121,NaN,NaN,Raro,só pdf,NaN,-,-
2,BUFONIDAE (sapos-cururu),"Rhinella mirandaribeiroi (Gallardo, 1965)",1,ok,X,X,"Santoro & Brandão, 2018",-,X,-,...,noturno,49-73,40-71,"janeiro, fevereiro, setembro e outubro","jan-fev, set-out",Raro,0.11606/issn.2176-7793.v40i1p1-73,https://scihub.to/10.11606/issn.2176-7793.v40i...,-,-


### Species name, id and Authorship

In [571]:
df['scientificName_withAuthorship']=df['scientificName']
df['scientificName'] = df['scientificName_withAuthorship'].apply(lambda x: re.findall( '^\w+(?: cf.| aff.)? \w+\.?', str(x))[0])

df['scientificNameAuthorship'] = df[['scientificName_withAuthorship','scientificName']].apply(lambda x: x[0][len(x[1]):], axis=1)
df['scientificNameAuthorship'] = df['scientificNameAuthorship'].apply(lambda x: re.findall( '[\w,\s\-&]+', str(x) )).apply(lambda x: ''.join(x).strip())

ids = df['scientificName'].str.replace('\s','-', regex=True).str.replace('[.]','', regex=True).str.lower()
df.insert(0, 'id', ids)

df.drop('scientificName_withAuthorship', axis=1, inplace=True)

/tmp/ipykernel_619375/2063698823.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df['scientificNameAuthorship'] = df[['scientificName_withAuthorship','scientificName']].apply(lambda x: x[0][len(x[1]):], axis=1)


In [572]:
df[['id','scientificName','scientificNameAuthorship']].head(3)

,id,scientificName,scientificNameAuthorship
0,allobates-goianus,Allobates goianus,"Bokermann, 1975"
1,rhaebo-guttatus,Rhaebo guttatus,"Schneider, 1799"
2,rhinella-mirandaribeiroi,Rhinella mirandaribeiroi,"Gallardo, 1965"


### Family name and id

In [573]:
df['family'] = df['family'].str.extract("([A-Za-z]+)")[0].apply(lambda x: x.capitalize())
df['family_id'] = df['family'].str.lower()

In [574]:
df[['family','family_id']].value_counts()

family           family_id      
Hylidae          hylidae            24
Leptodactylidae  leptodactylidae    17
Bufonidae        bufonidae           5
Odontophrynidae  odontophrynidae     5
Microhylidae     microhylidae        2
Aromobatidae     aromobatidae        1
Dendrobatidae    dendrobatidae       1
Siphonopidae     siphonopidae        1
Strabomantidae   strabomantidae      1
Name: count, dtype: int64

### Vernacular name

In [575]:
def extractVernacularNames(string):
    if pd.isna(string): string=''
    names = re.split('/|;|,',string)
    names = [ n.strip().replace(' ','-').capitalize() for n in names ]
    if len(names)==1 and names[0]=='':
        return []
    else:
        return names

df['vernacularNames'] = df['vernacularNames'].apply(extractVernacularNames)

In [576]:
df['vernacularNames'].head(3)

0    [Sapo-foguete-de-goiás]
1          [Cururu-de-couro]
2              [Sapo-cururu]
Name: vernacularNames, dtype: object

### Red list

In [577]:
df['Red List'] = df['Red List'].str.lower()
df.loc[ df['Red List']=='não avaliada','Red List']='ne'
df = pd.get_dummies(data=df, prefix='redlist', columns=['Red List'], dtype=int)

In [578]:
df[['redlist_dd','redlist_lc', 'redlist_ne'  ]].head(3)

,redlist_dd,redlist_lc,redlist_ne
0,1,0,0
1,0,1,0
2,0,0,1


### Endemicidade

In [579]:
df['endemic_cerrado'] = df['Endêmico (Cerrado)'].apply(lambda x: 1 if x=='sim' else 0)
df['endemic_chapada'] = df['Endêmico (Chapada)'].apply(lambda x: 1 if x=='sim' else 0)
df.drop('Endêmico (Chapada)', axis=1, inplace=True)
df.drop('Endêmico (Cerrado)', axis=1, inplace=True)

# Helpers
getInsideParentheses = lambda x: [ str.lower(e) for e in re.findall( '\((.{0,5})\)', str(x) ) ]

In [580]:
df[['endemic_cerrado','endemic_chapada']].head(3)

,endemic_cerrado,endemic_chapada
0,1,0
1,0,0
2,1,0


### Detectability

In [581]:
df['detectability'] = df['detectability']\
    .str.replace('[^\w]','',regex=True)\
    .str.replace('ê','e')\
    .str.lower()
    
subst_dict = {
    'muitofrequente': 'ff',
    'frequente':'f',
    'raro': 'r',
    'muitoraro': 'rr'
}

df['detectability'] = df['detectability'].apply(lambda x: subst_dict[x] if x is not np.NaN else x)
df = pd.get_dummies(df, prefix='detectability',columns=['detectability'], dtype=int)

In [582]:
df[['detectability_f','detectability_ff','detectability_r', 'detectability_rr']].head(3)

,detectability_f,detectability_ff,detectability_r,detectability_rr
0,0,0,0,1
1,0,0,1,0
2,0,0,1,0


### Poleiro

In [583]:
df['poleiro'] = df['Poleiro (tipical calling perch)* '].apply( getInsideParentheses )
df.drop('Poleiro (tipical calling perch)* ', axis=1, inplace=True)

poleiro_types = list(set( el for ls in df['poleiro'] for el in ls))

for tp in poleiro_types:
    df[f'tcp_{tp}'] = df['poleiro'].apply(lambda x: 1 if f'{tp}' in x else 0)
    
df.drop('poleiro', axis=1, inplace=True)

In [584]:
df[['tcp_gr','tcp_hb','tcp_lb','tcp_te','tcp_tr']].head(3)

,tcp_gr,tcp_hb,tcp_lb,tcp_te,tcp_tr
0,0,0,0,1,0
1,0,0,0,1,0
2,0,0,0,1,0


### Habitat breeding

In [585]:
matches = df['habitat_breeding'].str.lower().str.extractall('\((\w+)\)')
hb_dummies = pd.get_dummies(matches, prefix="habitat_breeding").groupby(level=0).sum()

hb_dummies = hb_dummies.reindex(df.index).fillna(0).astype(int)

df = pd.concat([df,hb_dummies], axis=1)
df.drop('habitat_breeding',axis=1, inplace=True)

In [586]:
df[['habitat_breeding_le','habitat_breeding_lo','habitat_breeding_tr']].head(3)

,habitat_breeding_le,habitat_breeding_lo,habitat_breeding_tr
0,0,0,1
1,1,0,0
2,1,0,0


### Phytophysiognomies categoricals

In [587]:
replaces = {
} 
df['phytos'] = df['phytos'].str.lower().str.strip().str.split('\s*,\s*')\
    .apply(lambda x: [replaces.get(i,i) for i in x] if isinstance(x,list) else '')\
    .str.join(',')

In [588]:
df['phytos'].head()

0             mg
1             mg
2       ce,ca,aa
3       mg,ce,aa
4    mg,ce,cr,aa
Name: phytos, dtype: object

### Atividade

In [589]:
df['atividade_diu'] = df['Atividade (noturno/diurno)'].apply(lambda x: 1 if 'diurno' in str(x) else 0)
df['atividade_not'] = df['Atividade (noturno/diurno)'].apply(lambda x: 1 if 'noturno' in str(x) else 0)
df.drop('Atividade (noturno/diurno)', axis=1, inplace=True)

In [590]:
df[['atividade_diu','atividade_not']].value_counts()

atividade_diu  atividade_not
0              1                49
1              0                 3
               1                 3
0              0                 2
Name: count, dtype: int64

### Tamanho

In [591]:
def calculaTamanhoMedio(strTamanho):
    # Recebe valores em string como '2.3-1.2'
    if type(strTamanho)!=str:
        return None
    listaMedidas = [ float(medida.strip().replace(',','.')) for medida in re.findall('([0-9/.,]+)',strTamanho) ]
    return sum(listaMedidas)/len(listaMedidas)

df['tamanho_femea_med'] = df['tamanho_femea'].apply(calculaTamanhoMedio)
df['tamanho_macho_med'] = df['tamanho_macho'].apply(calculaTamanhoMedio)
df['tamanho_med'] = (df['tamanho_macho_med']+df['tamanho_femea_med'])/2

In [592]:
df[['tamanho_femea','tamanho_macho','tamanho_femea_med','tamanho_macho_med','tamanho_med']].head(3)

,tamanho_femea,tamanho_macho,tamanho_femea_med,tamanho_macho_med,tamanho_med
0,16.8-17,16.8-17,16.9,16.9,16.90
1,174,121,174.0,121.0,147.50
2,49-73,40-71,61.0,55.5,58.25


### Meses de ocorrência

1. Extract month ranges from strings

In [593]:
def occurrence_months(m_start, m_end):
    d=['jan','fev','mar','abr','mai','jun','jul','ago','set','out','nov','dez']
    idx_mstart = d.index(m_start)
    idx_mend = d.index(m_end)
    
    if idx_mstart > idx_mend:
        return d[idx_mstart:]+d[:idx_mend+1]
    else:
        return d[idx_mstart:idx_mend+1]
    
    
def months_to_array(mths):
    d=['jan','fev','mar','abr','mai','jun','jul','ago','set','out','nov','dez']
    return [ 1 if i in mths else 0 for i in d ]

In [594]:
def monthsToArray(periods):
    if type(periods) is not list:
        return [0 for i in range(12)]
    
    d=['jan','fev','mar','abr','mai','jun','jul','ago','set','out','nov','dez']
    def getOcurrenceMonths(month_start, month_end):
        idx_mstart = d.index(month_start)
        idx_mend = d.index(month_end)
        if idx_mstart > idx_mend:
            return d[idx_mstart:]+d[:idx_mend+1]
        else:
            return d[idx_mstart:idx_mend+1]
        
    months = [ month for period in periods for month in getOcurrenceMonths(*period.strip().split('-')) ]
    return [1 if i in months else 0 for i in d]


df['month_vec'] = df['meses_ocorrencia'].str.split(',').apply(monthsToArray )

### Select fields to use

In [595]:
fieldsToUse = ["id", "family", "scientificName","vernacularNames",
"phytos", "scientificNameAuthorship", "family_id", "redlist_dd",
"redlist_lc","redlist_ne","endemic_cerrado", "endemic_chapada",
"detectability_f", "detectability_ff", "detectability_r", "detectability_rr",
"tcp_te","tcp_gr","tcp_tr","tcp_lb","tcp_hb",
"habitat_breeding_le","habitat_breeding_lo","habitat_breeding_tr",
"atividade_diu","atividade_not",
"tamanho_med","month_vec"]

df = df[fieldsToUse]

### Sort dataframe by Family and id

In [596]:
df = df.sort_values(by=['family','id'])

### Write to json

In [ ]:
species_data_path = '../../_data/species.json'

d = df.to_dict(orient='records')

with open(species_data_path, 'w') as f:
    json.dump(d, f,indent=1, ensure_ascii=False)
    
    
# TODO: Adicionar arquivo de dados também ao diretório onde fica o html da lista de espécies (p/ filtragem)

# Constrói classes de habitat